In [ ]:
import requests
import os
import time
import pandas as pd
import aiohttp
import asyncio
import json

import nest_asyncio
nest_asyncio.apply()

In [ ]:
TARGET = './mun/completeDatav3.xlsx'
df = pd.read_excel(TARGET)
df

In [ ]:
temp = []
responses = []

In [ ]:
for i, data in df.iterrows():
    lat = data['latitude']
    lon = data['longitude']
    date = str(data['date']).replace('/', '-')
    temp.append([lat, lon, date])

In [ ]:
async def fetch_data(session, url):
    async with session.get(url) as response:
        return await response.json()

async def main():
    urls = [f"https://api.sunrisesunset.io/json?lat={lat}&lng={lon}&date_start={date}&date_end={date}" for lat, lon, date in temp]
    global responses

    async with aiohttp.ClientSession() as session:
        tasks = [fetch_data(session, url) for url in urls]
        responses = await asyncio.gather(*tasks)
        

In [ ]:
asyncio.run(main())
responses = [d['results'][0] for d in responses]

df_api = pd.DataFrame.from_dict(responses)
df_api = df_api[['date', 'sunrise', 'sunset']]
df_api['date'] = df_api['date'].replace('-', '/', regex=True)
df_api

In [ ]:
df['sunrise'] = pd.to_datetime(df_api['sunrise'])
df['sunset'] = pd.to_datetime(df_api['sunset'])
df['durationOfDay'] = (df['sunset'] - df['sunrise']).dt.total_seconds()/3600
df.head()

In [ ]:
df['sunrise'] = df['sunrise'].dt.time
df['sunset'] = df['sunset'].dt.time
df

In [ ]:
df = df[['date', 'latitude', 'longitude', 'sunrise', 'sunset', 'durationOfDay']]
df.head()

In [ ]:
df.to_excel('./mun/v4-dataWithSunriseSunset.xlsx', index=False)